Code for analysis of network activity
@uthor: Raju Tomer

In [1]:
import monnet_utils as tl

In [2]:
import os, time
import glob
import skimage.external.tifffile as tff
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import caiman as cm
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour, inspect_correlation_pnr
import re
import pickle
import scipy as spy
import skimage.transform as skt
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd 
from scipy import ndimage as ndi
from skimage.transform import downscale_local_mean
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
#import community
#import pyintergraph
#from cdlib import algorithms
import scipy.io as sio
import multiprocessing as mp

import scipy

from scipy import stats

from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd


In [3]:
###  Core data location sections
### Load different batches of data (from different directories) as dictionary

def read_init_dat(data_set):
    if (data_set == 'CD1'):
        analysis_dir = r'X:\People\Raju\CD1'
        f_data_dir = r'Y:\CD1\Results'
        dict_im_lab = tl.load_pickle(f_data_dir,  r'dict_im_lab-CD1-SM.pickle')
        dict_in_fn = tl.load_pickle(f_data_dir, r'dict_in_fn-CD1-SM.pickle')
    elif (data_set == 'C57'):
        analysis_dir = r'X:\People\Raju\C57'
        f_data_dir = r'Y:\C57\Results'
        dict_im_lab = tl.load_pickle(f_data_dir,  r'dict_im_lab-C57.pickle')
        dict_in_fn = tl.load_pickle(f_data_dir, r'dict_in_fn-C57.pickle')
    elif (data_set == 'Df16'):
        analysis_dir = r'X:\People\Raju\DF16'
        f_data_dir = r'Y:\Data\Results'
        dict_im_lab = tl.load_pickle(f_data_dir,  r'dict_im_lab-DF16-.pickle')
        dict_in_fn = tl.load_pickle(f_data_dir, r'dict_in_fn-DF16.pickle')
    elif (data_set == r'Setd1'):
        analysis_dir = r'X:\People\Raju\Setd1'
        f_data_dir = r'Y:\Setd1\Results'
        dict_im_lab = tl.load_pickle(f_data_dir,  r'dict_im_lab-SETD1.pickle')
        dict_in_fn = tl.load_pickle(f_data_dir, r'dict_in_fn-SETD1.pickle')

    dict_dff_NsP = tl.load_pickle(analysis_dir, 'dict_dff_NSp.pickle')

    file_names_nsp_ids = sorted(glob.glob(os.path.join(analysis_dir, 'nsp_ids_key*')))
    dict_nsp_ids_fn = tl.construct_fn_dict(file_names_nsp_ids, key_extr_pat=r'.*nsp_ids_key(\d*)_(.*\.tif)\..*')
    file_names_cords_A = sorted(glob.glob(os.path.join(analysis_dir, 'cords_A_key*')))
    dict_cords_fn = tl.construct_fn_dict(file_names_cords_A, key_extr_pat=r'.*cords_A_key(\d*).*')
    dict_nsp_labs = {}
    dict_com_neuron = {}
    for k,f in dict_nsp_ids_fn.items():
#         print('Reading: ', k, f)
        dict_nsp_labs[k] = np.load(os.path.join(analysis_dir,f))
    for k,f in dict_cords_fn.items():
#         print('Reading: ', k, f)
        dict_com_neuron[k] = np.load(os.path.join(analysis_dir,f))

    dict_dff = tl.load_pickle(analysis_dir, 'dict_dff.pickle')
    dict_dff_zsc = tl.load_pickle(analysis_dir, 'dict_dff_zsc.pickle')

    return analysis_dir, dict_dff_NsP, dict_im_lab, dict_in_fn, dict_nsp_labs, dict_com_neuron, dict_dff, dict_dff_zsc


In [ ]:
### Read all the data

ad_C57, d_dffNsp_C57, d_labIm_C57, d_fn_C57, d_nspIDs_C57, d_com_C57, d_dff_C57, d_dffZsc_C57 = read_init_dat('C57')
print('C57 Done')

ad_CD1, d_dffNsp_CD1, d_labIm_CD1, d_fn_CD1, d_nspIDs_CD1, d_com_CD1, d_dff_CD1, d_dffZsc_CD1 = read_init_dat('CD1')
print('CD1 Done')


In [ ]:
#### Plotting Dff traces example ####
# for Fig 01 example in panel 1

begin_tp = 0 ### begin skip
end_tp = -1  ### -1 mean no skipping
dff_cut = 0.0
norm_corr = False
fig_dir = r'W:\People\Raju\NNet_Paper\Figures\Fig01_dat'
save_fig = False
# s_zsc_th = 5
# # zsc_th = 0.25
# do_gauss_smooth = True
# gauss_ker = 5
fps=30
max_zsc_int = .75
for k in d_dff_CD1.keys():
    if (k == 195):
        fn = d_fn_CD1[k]
        print(fn)
        nsp_ids = d_nspIDs_CD1[k]
        nsp_ids_filt = nsp_ids[nsp_ids>0]
        dat = np.copy(d_dff_CD1[k])
        im_lab = d_labIm_CD1[k]
        coms = d_com_CD1[k][nsp_ids>0,:]
        dist_pw = scipy.spatial.distance.pdist(coms, 'euclidean')
        dist_pw = scipy.spatial.distance.squareform(dist_pw)
    #         im_max = tff.imread(fn1)
        post_str = 'k' + str(k) + '_' + os.path.basename(fn) + '.png'
    ####### plot dff
        if (1):
            sz_per_neuron = 0.025
            gr_ht = np.maximum(1,int(dat.shape[0]*sz_per_neuron))
            fig, ax = plt.subplots(1, 1, figsize=(7/2,gr_ht/2), sharey=True) 
    #             dff_th = 1
    #             dat1 = dat1 - dff_th
    #             dat1[dat1<0] = 0
    #             dat2 = dat2 - dff_th
    #             dat2[dat2<0] = 0
            n_start = 240
            n_stop = 265
            tl.draw_activity_comp_in_range(dat, ax, nsp_ids_filt, max_zsc_int, 
                                             begin_tp = begin_tp, end_tp=end_tp, dff_bar=1,
                                           fps=30, n_start=n_start, n_stop=n_stop, lw=.55)
    #         tl.draw_activity_comp(dat2, ax2, nsp_ids_filt, max_zsc_int, 
    #                                          begin_tp = begin_tp, end_tp=end_tp, dff_bar=1, fps=30)
            plt.tight_layout()
            plt.show()

            if (save_fig):
                fig_fn = os.path.join(fig_dir,'dff_k' + str(k) + '_' + str(n_start) + '_' + str(n_stop) + '.png')
                print(fig_fn)
                fig.savefig(fig_fn, dpi=300, bbox_inches="tight")            
                fig_fn = os.path.join(fig_dir,'dff_k' + str(k) + '_' + str(n_start) + '_' + str(n_stop) + '.eps')
                print(fig_fn)
                fig.savefig(fig_fn, dpi=300, bbox_inches="tight")            


In [6]:
def calc_s_c(d_dff, p=2, dff_th=0.0):
    d_s = {}
    d_c = {}
    d_s_zsc = {}
    for k in d_dff.keys():
        if(k%50 == 0):
            print(k)
        dff = np.copy(d_dff[k])
        dff = dff + 0.0001
#         dff[dff<dff_th] = 0
        try:
            d_c[k], d_s[k] = tl.convert_f_to_cs(dff, p=p)
            d_s_zsc[k] = spy.stats.zscore(np.copy(d_s[k]), axis=1)
        except:
            print('Error in key: ', k)
            continue
    return d_c, d_s, d_s_zsc

if(1):
    d_dff_c_CD1_p2, d_dff_s_CD1_p2, d_dff_s_zsc_CD1_p2 = calc_s_c(d_dff_CD1, p=2, dff_th=0.0)
    tl.save_pickle(ad_CD1, 'd_dff_c_CD1_p2.pickle', d_dff_c_CD1_p2)
    tl.save_pickle(ad_CD1, 'd_dff_s_CD1_p2.pickle', d_dff_s_CD1_p2)
    tl.save_pickle(ad_CD1, 'd_dff_s_zsc_CD1_p2.pickle', d_dff_s_zsc_CD1_p2)
    print('Done1')

    d_dff_c_C57_p2, d_dff_s_C57_p2, d_dff_s_zsc_C57_p2 = calc_s_c(d_dff_C57, p=2, dff_th=0.0)
    tl.save_pickle(ad_C57, 'd_dff_c_C57_p2.pickle', d_dff_c_C57_p2)
    tl.save_pickle(ad_C57, 'd_dff_s_C57_p2.pickle', d_dff_s_C57_p2)
    tl.save_pickle(ad_C57, 'd_dff_s_zsc_C57_p2.pickle', d_dff_s_zsc_C57_p2)
    print('Done2')

    d_dff_c_Setd1_p2, d_dff_s_Setd1_p2, d_dff_s_zsc_Setd1_p2 = calc_s_c(d_dff_Setd1, p=2, dff_th=0.0)
    tl.save_pickle(ad_Setd1, 'd_dff_c_Setd1_p2.pickle', d_dff_c_Setd1_p2)
    tl.save_pickle(ad_Setd1, 'd_dff_s_Setd1_p2.pickle', d_dff_s_Setd1_p2)
    tl.save_pickle(ad_Setd1, 'd_dff_s_zsc_Setd1_p2.pickle', d_dff_s_zsc_Setd1_p2)
    print('Done3')

    d_dff_c_Df16_p2, d_dff_s_Df16_p2, d_dff_s_zsc_Df16_p2 = calc_s_c(d_dff_Df16, p=2, dff_th=0.0)
    tl.save_pickle(ad_Df16, 'd_dff_c_Df16_p2.pickle', d_dff_c_Df16_p2)
    tl.save_pickle(ad_Df16, 'd_dff_s_Df16_p2.pickle', d_dff_s_Df16_p2)
    tl.save_pickle(ad_Df16, 'd_dff_s_zsc_Df16_p2.pickle', d_dff_s_zsc_Df16_p2)
    print('Done4')



In [ ]:
if (0):   #Run one time only. Afterwards pickles are loaded as in lext if statement
    tl.save_pickle(ad_CD1, 'd_dff_c_CD1_p2.pickle', d_dff_c_CD1_p2)
    tl.save_pickle(ad_CD1, 'd_dff_s_CD1_p2.pickle', d_dff_s_CD1_p2)
    tl.save_pickle(ad_CD1, 'd_dff_s_zsc_CD1_p2.pickle', d_dff_s_zsc_CD1_p2)
    
    tl.save_pickle(ad_C57, 'd_dff_c_C57_p2.pickle', d_dff_c_C57_p2)
    tl.save_pickle(ad_C57, 'd_dff_s_C57_p2.pickle', d_dff_s_C57_p2)
    tl.save_pickle(ad_C57, 'd_dff_s_zsc_C57_p2.pickle', d_dff_s_zsc_C57_p2)

    tl.save_pickle(ad_Setd1, 'd_dff_c_Setd1_p2.pickle', d_dff_c_Setd1_p2)
    tl.save_pickle(ad_Setd1, 'd_dff_s_Setd1_p2.pickle', d_dff_s_Setd1_p2)
    tl.save_pickle(ad_Setd1, 'd_dff_s_zsc_Setd1_p2.pickle', d_dff_s_zsc_Setd1_p2)

    tl.save_pickle(ad_Df16, 'd_dff_c_Df16_p2.pickle', d_dff_c_Df16_p2)
    tl.save_pickle(ad_Df16, 'd_dff_s_Df16_p2.pickle', d_dff_s_Df16_p2)
    tl.save_pickle(ad_Df16, 'd_dff_s_zsc_Df16_p2.pickle', d_dff_s_zsc_Df16_p2)
    
if (1):
    d_dff_c_CD1_p2 = tl.load_pickle(ad_CD1, 'd_dff_c_CD1_p2.pickle')
    d_dff_s_CD1_p2 = tl.load_pickle(ad_CD1, 'd_dff_s_CD1_p2.pickle')
    d_dff_s_zsc_CD1_p2 = tl.load_pickle(ad_CD1, 'd_dff_s_zsc_CD1_p2.pickle')
    
    d_dff_c_C57_p2 = tl.load_pickle(ad_C57, 'd_dff_c_C57_p2.pickle')
    d_dff_s_C57_p2 = tl.load_pickle(ad_C57, 'd_dff_s_C57_p2.pickle')
    d_dff_s_zsc_C57_p2 = tl.load_pickle(ad_C57, 'd_dff_s_zsc_C57_p2.pickle')

    d_dff_c_Setd1_p2 = tl.load_pickle(ad_Setd1, 'd_dff_c_Setd1_p2.pickle')
    d_dff_s_Setd1_p2 = tl.load_pickle(ad_Setd1, 'd_dff_s_Setd1_p2.pickle')
    d_dff_s_zsc_Setd1_p2 = tl.load_pickle(ad_Setd1, 'd_dff_s_zsc_Setd1_p2.pickle')

    d_dff_c_Df16_p2 = tl.load_pickle(ad_Df16, 'd_dff_c_Df16_p2.pickle')
    d_dff_s_Df16_p2 = tl.load_pickle(ad_Df16, 'd_dff_s_Df16_p2.pickle')
    d_dff_s_zsc_Df16_p2 = tl.load_pickle(ad_Df16, 'd_dff_s_zsc_Df16_p2.pickle')
    
if (0):
    d_dff_C57_s = tl.load_pickle(ad_C57, 'dict_dff_s.pickle')
    d_dff_C57_s_zsc = tl.load_pickle(ad_C57, 'dict_dff_s_zsc.pickle')

    d_dff_CD1_s = tl.load_pickle(ad_CD1, 'dict_dff_s.pickle')
    d_dff_CD1_s_zsc = tl.load_pickle(ad_CD1, 'dict_dff_s_zsc.pickle')

    d_dff_Setd1_s = tl.load_pickle(ad_Setd1, 'dict_dff_s.pickle')
    d_dff_Setd1_s_zsc = tl.load_pickle(ad_Setd1, 'dict_dff_s_zsc.pickle')

    d_dff_Df16_s = tl.load_pickle(ad_Df16, 'dict_dff_s.pickle')
    d_dff_Df16_s_zsc = tl.load_pickle(ad_Df16, 'dict_dff_s_zsc.pickle')


In [ ]:
#### calculate firing fwhm
####fwhm_run

def run_fwhm_rise_tm(d_s_zsc, d_nsp, d_c, zsc_th=3, perc=0.25):
    d_rise_tm = {}
    d_rise_tm_pos = {}
    d_fwhm = {}
    d_fwhm_pos_back = {}
    d_fwhm_pos_fwd = {}
    d_no_spk_in_fwhm = {}
    for k in d_s_zsc.keys():
        print(k)
        nsp_ids = d_nsp[k]
        s_zsc = d_s_zsc[k]
        s_zsc = s_zsc[nsp_ids>0,:]
        c = d_c[k]
        c = c[nsp_ids>0,:]
        d_rise_tm[k], d_rise_tm_pos[k] = tl.calc_rise_tm(c, s_zsc, zsc_th=3)
        d_fwhm_pos_back[k], d_fwhm_pos_fwd[k], d_fwhm[k], d_no_spk_in_fwhm[k] = tl.calc_fwhm_spikes(c, s_zsc, zsc_th=zsc_th, perc=perc)
    return d_rise_tm, d_rise_tm_pos, d_fwhm_pos_back, d_fwhm_pos_fwd, d_fwhm, d_no_spk_in_fwhm
    

d_rise_tm_C57, d_rise_tm_pos_C57, d_fwhm_pos_back_C57, d_fwhm_pos_fwd_C57, d_fwhm_C57, d_no_spk_in_fwhm_C57 = run_fwhm_rise_tm(
    d_dff_s_zsc_C57_p2, d_nspIDs_C57, d_dff_C57, zsc_th=5, perc=0.75)
print('c57 done')

d_rise_tm_Setd1, d_rise_tm_pos_Setd1, d_fwhm_pos_back_Setd1, d_fwhm_pos_fwd_Setd1, d_fwhm_Setd1, d_no_spk_in_fwhm_Setd1 = run_fwhm_rise_tm(
    d_dff_s_zsc_Setd1_p2, d_nspIDs_Setd1, d_dff_Setd1, zsc_th=5, perc=0.75)
print('setd1 done')

d_rise_tm_CD1, d_rise_tm_pos_CD1, d_fwhm_pos_back_CD1, d_fwhm_pos_fwd_CD1, d_fwhm_CD1, d_no_spk_in_fwhm_CD1 = run_fwhm_rise_tm(
    d_dff_s_zsc_CD1_p2, d_nspIDs_CD1, d_dff_CD1, zsc_th=5, perc=0.75)
print('CD1 done')


In [17]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000

csv_dir = r'Y:\Data'
dd_CD1 = pd.read_csv(os.path.join(csv_dir, r'WT experiments list CD1 v02.csv'))
dd_C57 = pd.read_csv(os.path.join(csv_dir, r'WT experiments list C57 v02.csv'))

csv_dir1 = r'Y:\Data'
dd_Setd1 = pd.read_csv(os.path.join(csv_dir1, r'SETD1 experiments v02.csv'))


csv_dir2 = r'Y:\Data'
dd_Df16 = pd.read_csv(os.path.join(csv_dir2, r'DF16 experiment v01.csv'))


In [19]:
### Firing rate calculations
def calc_frpm(d_s_zsc, d_nsp, zsc_th=5):
    d_frpm = {}
    for k in d_s_zsc.keys():
        s_zsc = np.copy(d_s_zsc[k])
        nsp_ids = d_nsp[k]
        s_zsc = s_zsc[nsp_ids>0,:]
        e = s_zsc >= zsc_th
        s_zsc[e] = 1
        s_zsc[~e] = 0
        s_sum = np.sum(s_zsc, axis=1)
        s_sum = s_sum*30*60/s_zsc.shape[1]
#         print(np.sum(s_sum)/s_zsc.shape[0])
        d_frpm[k] = np.sum(s_sum)/s_zsc.shape[0]
    return d_frpm

d_frpm_CD1 = calc_frpm(d_dff_s_zsc_CD1_p2, d_nspIDs_CD1)
d_frpm_C57 = calc_frpm(d_dff_s_zsc_C57_p2, d_nspIDs_C57)
d_frpm_Setd1 = calc_frpm(d_dff_s_zsc_Setd1_p2, d_nspIDs_Setd1)
d_frpm_Df16 = calc_frpm(d_dff_s_zsc_Df16_p2, d_nspIDs_Df16)


In [20]:
d_frpm_in_DIVs_CD1 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_CD1, d_frpm_CD1)
d_frpm_in_DIVs_CD1_in_C57 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_CD1_in_C57, d_frpm_C57)
d_frpm_in_DIVs_C57 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_C57, d_frpm_C57)
d_frpm_in_DIVs_Setd1 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_Setd1, d_frpm_Setd1)
d_frpm_in_DIVs_Df16 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_Df16, d_frpm_Df16)


DIV:  11
DIV:  12
DIV:  13
DIV:  14
DIV:  15
DIV:  16
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  28
DIV:  29
DIV:  25
DIV:  16
DIV:  17
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  25
DIV:  26
DIV:  28
DIV:  29
DIV:  30
DIV:  31
DIV:  32
DIV:  33
DIV:  16
DIV:  17
DIV:  18
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  26
DIV:  28
DIV:  30
DIV:  31
DIV:  33
DIV:  16
DIV:  20
DIV:  21
DIV:  23
DIV:  24
DIV:  32
DIV:  35


In [157]:
def sort_into_DIVs(d_k_in_DIVs, d_prop, max_tm=4000):
    d_prop_in_DIVs = {}
    for dk in d_k_in_DIVs.keys():
        print('DIV: ', dk)
        k_in_dk = d_k_in_DIVs[dk]
        list_vals = []
        for k in k_in_dk:
            d = d_prop[k]
            mn_list = []
            for k1 in d.keys():
#                 print(np.max(d[k1]))
                if (np.max(d[k1]) <= max_tm):
                    mn_list.append(np.mean(d[k1]))
            list_vals.append(np.mean(mn_list))
        d_prop_in_DIVs[dk] = list_vals
    return d_prop_in_DIVs

d_fwhm_in_DIVs_CD1 = sort_into_DIVs(d_k_in_DIVs_CD1, d_fwhm_CD1, max_tm=4000)
d_fwhm_in_DIVs_CD1_in_C57 = sort_into_DIVs(d_k_in_DIVs_CD1_in_C57, d_fwhm_C57, max_tm=4000)
d_fwhm_in_DIVs_C57 = sort_into_DIVs(d_k_in_DIVs_C57, d_fwhm_C57, max_tm=4000)
d_fwhm_in_DIVs_Setd1 = sort_into_DIVs(d_k_in_DIVs_Setd1, d_fwhm_Setd1, max_tm=4000)
d_fwhm_in_DIVs_Df16 = sort_into_DIVs(d_k_in_DIVs_Df16, d_fwhm_Df16, max_tm=4000)



DIV:  11
DIV:  12
DIV:  13
DIV:  14
DIV:  15
DIV:  16
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  28
DIV:  29
DIV:  25
DIV:  16
DIV:  17
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  25
DIV:  26
DIV:  28
DIV:  29
DIV:  30
DIV:  31
DIV:  32
DIV:  33
DIV:  16
DIV:  17
DIV:  18
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  26
DIV:  28
DIV:  30
DIV:  31
DIV:  33
DIV:  16
DIV:  20
DIV:  21
DIV:  23
DIV:  24
DIV:  32
DIV:  35
DIV:  11
DIV:  12
DIV:  13
DIV:  14
DIV:  15
DIV:  16
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  28
DIV:  29
DIV:  25
DIV:  16
DIV:  17
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  25
DIV:  26
DIV:  28
DIV:  29
DIV:  30
DIV:  31
DIV:  32
DIV:  33
DIV:  16
DIV:  17
DIV:  18
DIV:  19
DIV:  20
DIV:  21
DIV:  22
DIV:  23
DIV:  24
DIV:  26
DIV:  28
DIV:  30
DIV:  31
DIV:  33


In [ ]:
#### Correlations calculations for All

if(1):
    print('#### CD1')
    d_mn_pwc_CD1, d_mn_pwc_CD1_inter, d_mn_pwc_CD1_intra = tl.calc_pwc_mn(d_k_in_DIVs_CD1, d_dff_CD1,
                                                                          d_nspIDs_CD1, dff_cut=0.0, norm_corr = False)

    print('#### C57')
    d_mn_pwc_C57, d_mn_pwc_C57_inter, d_mn_pwc_C57_intra = tl.calc_pwc_mn(d_k_in_DIVs_C57, d_dff_C57, 
                                                                       d_nspIDs_C57, dff_cut=0.0, norm_corr = False)
    print('#### Setd1')
    d_mn_pwc_Setd1, d_mn_pwc_Setd1_inter, d_mn_pwc_Setd1_intra = tl.calc_pwc_mn(d_k_in_DIVs_Setd1, d_dff_Setd1, 
                                                                       d_nspIDs_Setd1, dff_cut=0.0, norm_corr = False)
    print('#### Df16')
    d_mn_pwc_Df16, d_mn_pwc_Df16_inter, d_mn_pwc_Df16_intra = tl.calc_pwc_mn(d_k_in_DIVs_Df16, d_dff_Df16, 
                                                                       d_nspIDs_Df16, dff_cut=0.0, norm_corr = False)

In [ ]:
def print_star(p):
    if(p<0.0001):
        return '****'
    elif(p<0.001):
        return '***'
    elif(p<0.01):
        return '**'
    elif(p<0.05):
        return '*'
    else:
        return 'n.s.'

In [ ]:
###Phase
### divide DIVs into 3 phases (pools) 
to_pool1 = list(np.arange(0,18))
to_pool2 = list(np.arange(18,27))
to_pool3 = list(np.arange(27,30))

d_1 = d_mn_pwc_CD1_inter.copy()
A = set(d_1.keys())

dat1 = []
dat2 = []
dat3 = []

for i in A:
    if (i in to_pool1):
        dat1.extend(d_1[i])
    if (i in to_pool2):
        dat2.extend(d_1[i])
    if (i in to_pool3):
        dat3.extend(d_1[i])
        
df_dat = []
for ii in range(len(dat1)):
    df_dat.append({'c':dat1[ii], 'state': 'p1'})
for ii in range(len(dat2)):
    df_dat.append({'c':dat2[ii], 'state': 'p2'})
for ii in range(len(dat3)):
    df_dat.append({'c':dat3[ii], 'state': 'p3'})

tdf = pd.DataFrame(df_dat)
tdf.to_pickle(r'E:\Raju\MoNNet\Global_pwc_tdf_01.pkl')

print('Global PWC pvals')
an = f_oneway(dat1, dat2, dat3)
print(an)
tukey = pairwise_tukeyhsd(endog=tdf['c'],
                          groups=tdf['state'],
                          alpha=0.05)
print('Tukey')
print(tukey)
print(print_star(tukey.pvalues[0]))
print(print_star(tukey.pvalues[1]))
print(print_star(tukey.pvalues[2]))

print('p1 mean:', np.mean(dat1), ' , p1 std:', np.std(dat1))
print('p2 mean:', np.mean(dat2), ' , p2 std:', np.std(dat2))
print('p3 mean:', np.mean(dat3), ' , p3 std:', np.std(dat3))
print('\n')
print('pairwise ttest')
t=stats.ttest_ind(dat1, dat2)
print('p1 and p2 pval: ', t.pvalue, print_star(t.pvalue))

t=stats.ttest_ind(dat2, dat3)
print('p2 and p3 pval: ', t.pvalue, print_star(t.pvalue))

t=stats.ttest_ind(dat1, dat3)
print('p1 and p3 pval: ', t.pvalue, print_star(t.pvalue))

fig = plt.figure(figsize=(2,3))
ax = fig.add_axes([0., 0., 1., 1.])
ax.margins(0.008)

ax = sns.barplot(x="state", y="c", data=tdf, ax=ax)
plt.ylim(0,1.05)
# plt.xticks(np.arange(10, 31, step=5))
plt.yticks(np.arange(0, 1.1, step=.5))

fig_dir = r'E:\Raju\MoNNet\F1Rel\phases_final'
save_fig = True
fig_fn = os.path.join(fig_dir,'CD1_PWC_Inter_phases')

if (save_fig):
    print(fig_fn + '.png')
    fig.savefig(fig_fn + '.png', dpi=200, bbox_inches="tight")            
    print(fig_fn + '.eps')
    fig.savefig(fig_fn + '.eps', dpi=200, bbox_inches="tight")            


In [ ]:
############# Pairwise Correlations plots ############
#### Inter ###

pdeg = 4
lw = 1
lwp = .5
psz=2
fig = plt.figure(figsize=(2,2))
ax = fig.add_axes([0., 0., 1., 1.])
ax.margins(0.008)
fig_dir = r'E:\Raju\MoNNet\Setd1FigRel'
save_fig = False
fig_fn = os.path.join(fig_dir,'CD1_C57_Setd1_Df16_PWC_Inter_pdeg'+ str(pdeg))

                      
                      
print(d_mn_pwc_CD1_inter.keys())
d, d_std = tl.gen_mn_of_mn(d_mn_pwc_CD1_inter)
d = tl.sort_dict(d)
d_std = tl.sort_dict(d_std)
ax.errorbar(d.keys(), d.values(), yerr=d_std.values(), fmt='o', 
            color='gray', label='CD1 WT', linewidth=lwp, markersize=psz)
x,y = tl.gen_poly_fit(d,deg=pdeg)
ax.plot(x, y, color='gray', linewidth=lw)

d, d_std = tl.gen_mn_of_mn(d_mn_pwc_C57_inter)
d = tl.sort_dict(d)
d_std = tl.sort_dict(d_std)
ax.errorbar(d.keys(), d.values(), yerr=d_std.values(), fmt='o', 
            label='C57 WT', color='C0', linewidth=lwp, markersize=psz)

x,y = tl.gen_poly_fit(d,deg=pdeg)
ax.plot(x, y, color='C0', linewidth=lw)



d, d_std = tl.gen_mn_of_mn(d_mn_pwc_Setd1_inter)
d = tl.sort_dict(d)
d_std = tl.sort_dict(d_std)
ax.errorbar(d.keys(), d.values(), yerr=d_std.values(), fmt='o', 
            label='Setd1', color='C1', linewidth=lwp, markersize=psz)
x,y = tl.gen_poly_fit(d,deg=pdeg)
ax.plot(x, y, color='C1', linewidth=lw)



d, d_std = tl.gen_mn_of_mn(d_mn_pwc_Df16_inter)
d = tl.sort_dict(d)
d_std = tl.sort_dict(d_std)
ax.errorbar(d.keys(), d.values(), yerr=d_std.values(), fmt='o', 
            label='Df16', color='green', linewidth=lwp, markersize=psz)
x,y = tl.gen_poly_fit(d,deg=pdeg)
ax.plot(x, y, color='green', linewidth=lw)




# plt.legend(loc=0)
plt.xlim(10,36)
plt.ylim(0,1.05)
plt.xticks(np.arange(10, 36, step=5))
# plt.grid()
# ax.set_xlabel('DIVs', fontsize=18)
# ax.set_ylabel('Avg Pairwise Correlation', fontsize=18)
plt.show()

if (save_fig):
    print(fig_fn + '.png')
    fig.savefig(fig_fn + '.png', dpi=200, bbox_inches="tight")            
    print(fig_fn + '.eps')
    fig.savefig(fig_fn + '.eps', dpi=200, bbox_inches="tight")            


In [ ]:
############ Modularity analysis
if(1):
    ccut = 0.5
#     pool = mp.Pool(processes=5)
    print('#### CD1')
    d_no_comm_CD1, d_mod_CD1 = tl.calc_mod_comm(d_k_in_DIVs_CD1, d_dff_CD1,
                                             d_nspIDs_CD1, d_com_CD1, dff_cut=0.1, norm_corr = False, ccut=ccut, binary=False)

    print('#### C57')
    d_no_comm_C57, d_mod_C57 = tl.calc_mod_comm(d_k_in_DIVs_C57, d_dff_C57,
                                             d_nspIDs_C57, d_com_C57, dff_cut=0.1, norm_corr = False, ccut=ccut, binary=False)

    print('#### Setd1')
    d_no_comm_Setd1, d_mod_Setd1 = tl.calc_mod_comm(d_k_in_DIVs_Setd1, d_dff_Setd1,
                                             d_nspIDs_Setd1, d_com_Setd1, dff_cut=0.1, norm_corr = False, ccut=ccut, binary=False)

    print('#### Df16')
    d_no_comm_Df16, d_mod_Df16 = tl.calc_mod_comm(d_k_in_DIVs_Df16, d_dff_Df16,
                                             d_nspIDs_Df16, d_com_Df16, dff_cut=0.1, norm_corr = False, ccut=ccut, binary=False)



In [ ]:
### save eps for short list
### for Setd1
##### Plot graphs with labelled communities and save them, limited number to 9 distinct colors/clusters
if(1): 
    fig_dir = r'X:\People\Raju\F1Rel\selected_graphs'
    save_fig = True

    louvian = True
    node_size = 12
    d_nspIDs = d_nspIDs_CD1
    d_dff = d_dff_CD1
    d_labIm = d_labIm_CD1
    d_com = d_com_CD1
    d_k_in_DIVs = d_k_in_DIVs_CD1
    cmap_ = plt.cm.Blues
    dff_cut = 0.1
    edge_wt = 1
    alpha=0.75
    vmin = 0.3
    vmax = 1.0
    for dk in d_k_in_DIVs.keys():
        print('DIV: ', dk)
        k_in_dk = d_k_in_DIVs[dk]
        list_mod = []
        list_no_comm = []
        for k in k_in_dk:
#             if (k in [30, 70,170, 268]):
            if (k in [70, 170, 276]):
                print('DIV: ', dk, ', key: ', k)
                tmp_dat = np.copy(d_dff[k])
                nsp_ids = d_nspIDs[k]
                nsp_ids_uniq = np.unique(nsp_ids)
                if (0 in nsp_ids_uniq):
                    tmp_dat_filt = tmp_dat[nsp_ids>0,:]
                    nsp_ids_filt = nsp_ids[nsp_ids>0]
                else:
                    tmp_dat_filt = np.copy(tmp_dat)
                    nsp_ids_filt = np.copy(nsp_ids)
                tmp_dat_filt[tmp_dat_filt<dff_cut] = 0
                r = np.corrcoef(tmp_dat_filt)
                r = np.nan_to_num(r)
                np.fill_diagonal(r,0)

                im_lab = d_labIm[k]
                coms = d_com[k]
                coms = coms[nsp_ids>0,:]
                index = list(range(1,coms.shape[0]+1))
                cord = {}
                for i in range(1,coms.shape[0]+1):
                    cord[i] = [coms[i-1,1],coms[i-1,0]]

                g=nx.Graph()
                for i in range(len(index)):
                    g.add_node(index[i])
                for i in range(len(index)):
                    for j in range(i+1, len(index)):
                        if (r[i,j] > 0):
                            g.add_edge(index[i],index[j],weight=r[i,j])

                if (louvian):
                    clusters = community.best_partition(g)
                    size = float(len(set(clusters.values())))
                    mod = community.modularity(clusters, g, weight='weight')
                else:
                    gi = pyintergraph.nx2igraph(g)
                    clusters = community.best_partition(gi)
                    a = clusters.communities
                    size = len(a)
                    mod = 0
                print('No of clusters: ', len(set(clusters.values())))
                colors = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
                fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 9), sharex=True, sharey=True)
                im_lab_ = np.copy(im_lab)
                im_lab_[im_lab_>0] = 10
                ax[0].imshow(im_lab_, cmap='Greys', vmin=0, vmax=40)
                ax[1].imshow(im_lab_, cmap='Greys', vmin=0, vmax=40)

                c_id = 0
                for comm in set(clusters.values()):
                    list_nodes = [nodes for nodes in clusters.keys()
                                                if clusters[nodes] == comm]
                    if (len(list_nodes)==1):
                        nx.draw_networkx_nodes(g, cord, list_nodes, node_size = node_size,
                                                    node_color = 'black', ax=ax[0])
                    if (len(list_nodes)>1):
                    #     print(list_nodes)
                        nx.draw_networkx_nodes(g, cord, list_nodes, node_size = node_size,
                                                    node_color = colors[c_id], ax=ax[0])
                        c_id = c_id + 1
                        if(c_id>8):
                            c_id=0
                            print('More than 9 clusters!')
                # plot edges graph
                g_edges = sorted(g.edges(data=True), key=lambda t: t[2].get('weight', 1))
                g_colors = []
                for e in g_edges:
                    c = cmap_((e[2]['weight'] - vmin)/(vmax - vmin))
                    g_colors.append(c)
                    nx.draw_networkx_edges(g,pos=cord, edgelist=[e], with_labels=False, 
                                           width=edge_wt, edge_color=c, alpha=alpha, ax=ax[1], node_size=node_size)
    #             nx.draw_networkx_nodes(g,pos=cord,node_size=node_size, ax=ax[1])
                c_id=0
                for comm in set(clusters.values()):
                    list_nodes = [nodes for nodes in clusters.keys()
                                                if clusters[nodes] == comm]
                    if (len(list_nodes)==1):
                        nx.draw_networkx_nodes(g, cord, list_nodes, node_size = node_size,
                                                    node_color = 'black', ax=ax[1])
                    if (len(list_nodes)>1):
                    #     print(list_nodes)
                        nx.draw_networkx_nodes(g, cord, list_nodes, node_size = node_size,
                                                    node_color = colors[c_id], ax=ax[1])
                        c_id = c_id + 1
                        if(c_id>8):
                            c_id=0
                            print('More than 9 clusters!')

                colors_unscaled=[tuple(map(lambda x: (vmax - vmin)*x + vmin, y)) for y in g_colors]
                plt.ioff()
                fig_temp,ax_temp = plt.subplots()
                heatmap = plt.pcolor(colors_unscaled,cmap=cmap_, vmin=vmin, vmax=vmax)
                plt.close(fig_temp)
                plt.ion()
                cbar = plt.colorbar(heatmap, ax=ax, shrink=0.5, use_gridspec=True)
                cbar.ax.set_ylabel('Corr',labelpad=15,rotation=270)
                cbar.set_ticks(np.arange(vmin, vmax+1, 0.1))

                plt.show()
                if (save_fig):
                    nc = len(set(clusters.values()))
                    fig_fn = os.path.join(fig_dir,'DIV' + str(dk) + '_Louv_k' + str(k) + '_nc' 
                                          + str(nc) + '.png')
                    print(fig_fn)
                    fig.savefig(fig_fn, dpi=150, bbox_inches="tight")
                    fig_fn = os.path.join(fig_dir,'DIV' + str(dk) + '_Louv_k' + str(k) + '_nc' 
                                          + str(nc) + '.eps')
                    fig.savefig(fig_fn, dpi=150, bbox_inches="tight", format='eps')


In [ ]:
### generate graphs (for network efficiency calculations) for all and store as dict files
if(1):
    d_g_CD1 = tl.gen_graph_collection(d_k_in_DIVs_CD1, d_dff_CD1, 
                                   d_nspIDs_CD1, d_com_CD1, dff_cut=0.05, norm_corr = False, ccut=0.5, binary=False)

    d_g_C57 = tl.gen_graph_collection(d_k_in_DIVs_C57, d_dff_C57, 
                                   d_nspIDs_C57, d_com_C57, dff_cut=0.05, norm_corr = False, ccut=0.5, binary=False)

    d_g_Setd1 = tl.gen_graph_collection(d_k_in_DIVs_Setd1, d_dff_Setd1, 
                                   d_nspIDs_Setd1, d_com_Setd1, dff_cut=0.05, norm_corr = False, ccut=0.5, binary=False)

    d_g_Df16 = tl.gen_graph_collection(d_k_in_DIVs_Df16, d_dff_Df16, 
                                   d_nspIDs_Df16, d_com_Df16, dff_cut=0.05, norm_corr = False, ccut=0.5, binary=False)


In [ ]:
### Global Network Efficiency
pool = mp.Pool(processes=96)
pres_geff_CD1 = [pool.apply(tl.calc_global_eff_worker, args=(k,d_g_CD1[k])) for k in d_g_CD1.keys()]
print(pres_geff_CD1)
print('Done')
pool.close()
pool.join()

pool = mp.Pool(processes=96)
pres_geff_CD1_in_C57 = [pool.apply(tl.calc_global_eff_worker, args=(k,d_g_CD1_in_C57[k])) for k in d_g_CD1_in_C57.keys()]
print(pres_geff_CD1_in_C57)
print('Done')
pool.close()
pool.join()

pool = mp.Pool(processes=96)
pres_geff_C57 = [pool.apply(tl.calc_global_eff_worker, args=(k,d_g_C57[k])) for k in d_g_C57.keys()]
print(pres_geff_C57)
print('Done')
pool.close()
pool.join()

pool = mp.Pool(processes=96)
pres_geff_Setd1 = [pool.apply(tl.calc_global_eff_worker, args=(k,d_g_Setd1[k])) for k in d_g_Setd1.keys()]
print(pres_geff_Setd1)
print('Done')
pool.close()
pool.join()

pool = mp.Pool(processes=96)
pres_geff_Df16 = [pool.apply(tl.calc_global_eff_worker, args=(k,d_g_Df16[k])) for k in d_g_Df16.keys()]
print(pres_geff_Df16)
print('Done')
pool.close()
pool.join()


### Local Network efficiency
if (pool):
    pool.close()
    pool.join()
pool = mp.Pool(processes=80)
pres_leff_CD1 = [pool.apply(tl.calc_local_eff_worker, args=(k,d_g_CD1[k])) for k in d_g_CD1.keys()]
print(pres_leff_CD1)
print('Done')
pool.close()
pool.join()


if (pool):
    pool.close()
    pool.join()
pool = mp.Pool(processes=80)
pres_leff_C57 = [pool.apply(tl.calc_local_eff_worker, args=(k,d_g_C57[k])) for k in d_g_C57.keys()]
print(pres_leff_C57)
print('Done')
pool.close()
pool.join()


if (pool):
    pool.close()
    pool.join()
pool = mp.Pool(processes=80)
pres_leff_Setd1 = [pool.apply(tl.calc_local_eff_worker, args=(k,d_g_Setd1[k])) for k in d_g_Setd1.keys()]
print(pres_leff_Setd1)
print('Done')
pool.close()
pool.join()


if (pool):
    pool.close()
    pool.join()
pool = mp.Pool(processes=80)
pres_leff_Df16 = [pool.apply(tl.calc_local_eff_worker, args=(k,d_g_Df16[k])) for k in d_g_Df16.keys()]
print(pres_leff_Df16)
print('Done')
pool.close()
pool.join()



In [38]:
### save the calculated values
### this needs fixing

if (0):
    tl.save_pickle(ad_CD1, 'pres_leff_CD1_ccut05_weighted.pickle', pres_leff_CD1)
    tl.save_pickle(ad_C57, 'pres_leff_C57_ccut05_weighted.pickle', pres_leff_C57)
    tl.save_pickle(ad_Setd1, 'pres_leff_Setd1_ccut05_weighted.pickle', pres_leff_Setd1)
    tl.save_pickle(ad_Df16, 'pres_leff_Df16_ccut05_weighted.pickle', pres_leff_Df16)

if (0):
    tl.save_pickle(ad_CD1, 'pres_geff_CD1_ccut05_weighted.pickle', pres_geff_CD1)
    tl.save_pickle(ad_C57, 'pres_geff_C57_ccut05_weighted.pickle', pres_geff_C57)
    tl.save_pickle(ad_Setd1, 'pres_geff_Setd1_ccut05_weighted.pickle', pres_geff_Setd1)
    tl.save_pickle(ad_Df16, 'pres_geff_Df16_ccut05_weighted.pickle', pres_geff_Df16)

if(1):
    pres_leff_CD1 = tl.load_pickle(ad_CD1, 'pres_leff_CD1_ccut08.pickle')
    pres_leff_C57 = tl.load_pickle(ad_C57, 'pres_leff_C57_ccut08.pickle')
    pres_leff_Setd1 = tl.load_pickle(ad_Setd1, 'pres_leff_Setd1_ccut08.pickle')
    pres_leff_Df16 = tl.load_pickle(ad_Df16, 'pres_leff_Df16_ccut08.pickle')
    print('Done')
    
if(1):
    pres_geff_CD1 = tl.load_pickle(ad_CD1, 'pres_geff_CD1_ccut08.pickle')
    pres_geff_C57 = tl.load_pickle(ad_C57, 'pres_geff_C57_ccut08.pickle')
    pres_geff_Setd1 = tl.load_pickle(ad_Setd1, 'pres_geff_Setd1_ccut08.pickle')
    pres_geff_Df16 = tl.load_pickle(ad_Df16, 'pres_geff_Df16_ccut08.pickle')
    print('Done')

if(0):
    pres_leff_CD1 = tl.load_pickle(ad_CD1, 'pres_leff_CD1_ccut05_weighted.pickle')
    pres_leff_C57 = tl.load_pickle(ad_C57, 'pres_leff_C57_ccut05_weighted.pickle')
    pres_leff_Setd1 = tl.load_pickle(ad_Setd1, 'pres_leff_Setd1_ccut05_weighted.pickle')
    pres_leff_Df16 = tl.load_pickle(ad_Df16, 'pres_leff_Df16_ccut05_weighted.pickle')
    
if(0):
    pres_geff_CD1 = tl.load_pickle(ad_CD1, 'pres_geff_CD1_ccut05_weighted.pickle')
    pres_geff_C57 = tl.load_pickle(ad_C57, 'pres_geff_C57_ccut05_weighted.pickle')
    pres_geff_Setd1 = tl.load_pickle(ad_Setd1, 'pres_geff_Setd1_ccut05_weighted.pickle')
    pres_geff_Df16 = tl.load_pickle(ad_Df16, 'pres_geff_Df16_ccut05_weighted.pickle')
    

Done
Done


In [ ]:
### pool values back from pool

d_geff_CD1 = tl.convert_pool_dict(pres_geff_CD1)    
d_geff_in_DIVs_CD1 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_CD1, d_geff_CD1)

d_geff_C57 = tl.convert_pool_dict(pres_geff_C57)    
d_geff_in_DIVs_C57 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_C57, d_geff_C57)

d_geff_Setd1 = tl.convert_pool_dict(pres_geff_Setd1)    
d_geff_in_DIVs_Setd1 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_Setd1, d_geff_Setd1)

d_geff_Df16 = tl.convert_pool_dict(pres_geff_Df16)    
d_geff_in_DIVs_Df16 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_Df16, d_geff_Df16)


### Plot global efficiency, pool values back from pool

d_leff_CD1 = tl.convert_pool_dict(pres_leff_CD1)    
d_leff_in_DIVs_CD1 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_CD1, d_leff_CD1)

d_leff_C57 = tl.convert_pool_dict(pres_leff_C57)    
d_leff_in_DIVs_C57 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_C57, d_leff_C57)

d_leff_Setd1 = tl.convert_pool_dict(pres_leff_Setd1)    
d_leff_in_DIVs_Setd1 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_Setd1, d_leff_Setd1)

d_leff_Df16 = tl.convert_pool_dict(pres_leff_Df16)    
d_leff_in_DIVs_Df16 = tl.sort_graph_props_in_DIVs(d_k_in_DIVs_Df16, d_leff_Df16)


In [458]:
# Dimensionality
# modularity
# firing rates
def calc_pca(dict_dat, dict_nsp_ids, dff_cut=0.05, min_rows=3):
    d_pca_tm = {}
#     d_pca_sp = {}
    for k in sorted(dict_nsp_ids.keys()):
        tmp_dat = np.copy(dict_dat[k])
        tmp_dat = np.nan_to_num(tmp_dat)
        if (tmp_dat.shape[0] > min_rows):
            if(k%50==0):
                print(k)
            nsp_ids = dict_nsp_ids[k]
            nsp_ids_uniq = np.unique(nsp_ids)
            if (0 in nsp_ids_uniq):
                tmp_dat = tmp_dat[nsp_ids>0,:]
                nsp_ids = nsp_ids[nsp_ids>0]
            tmp_dat[tmp_dat<dff_cut] = 0
#             pca = PCA()
#             pca.fit(np.copy(tmp_dat))
#             d_pca_sp[k] = pca
            
            tmp_dat = tmp_dat.T
            pca = PCA()
            pca.fit(np.copy(tmp_dat))
            d_pca_tm[k] = pca
        else:
            print('Error in ', k, ' No of rows smaller than ', min_rows)
    return d_pca_tm

def get_pca_dims_var(d_k_in_DIVs, d_pca, cut=0.95, dim=0):
    """
    cut and dim are independet. cut is for dims, dim is for var
    """
    d_pca_dims = {}
    d_pca_var = {}
    for dk in d_k_in_DIVs.keys():
#         print('DIV: ', dk)
        k_in_dk = d_k_in_DIVs[dk]
        list_vals_dims = []
        list_vals_var = []
        for k in k_in_dk:
            var_r = d_pca[k].explained_variance_ratio_
            cumsum_var_r = np.cumsum(var_r)
            list_vals_dims.append(sum(cumsum_var_r <= cut))
            list_vals_var.append(var_r[dim])
#         print(list_vals_dims)
#         print(list_vals_var)
        d_pca_dims[dk] = np.array(list_vals_dims)
        d_pca_var[dk] = np.array(list_vals_var)
    return d_pca_dims, d_pca_var




In [459]:
if(1):
    d_pca_CD1 = calc_pca(d_dff_CD1, d_nspIDs_CD1, dff_cut=0.0, min_rows=3)
    d_pca_C57 = calc_pca(d_dff_C57, d_nspIDs_C57, dff_cut=0.0, min_rows=3)
    d_pca_Setd1 = calc_pca(d_dff_Setd1, d_nspIDs_Setd1, dff_cut=0.0, min_rows=3)
    d_pca_Df16 = calc_pca(d_dff_Df16, d_nspIDs_Df16, dff_cut=0.0, min_rows=3)


0
50


In [466]:
cut = 0.95
dim = 0
d_pca_dims_CD1, d_pca_var_CD1 = get_pca_dims_var(d_k_in_DIVs_CD1, d_pca_CD1, cut=cut, dim=dim)
d_pca_dims_C57, d_pca_var_C57 = get_pca_dims_var(d_k_in_DIVs_C57, d_pca_C57, cut=cut, dim=dim)
d_pca_dims_Setd1, d_pca_var_Setd1 = get_pca_dims_var(d_k_in_DIVs_Setd1, d_pca_Setd1, cut=cut, dim=dim)
d_pca_dims_Df16, d_pca_var_Df16 = get_pca_dims_var(d_k_in_DIVs_Df16, d_pca_Df16, cut=cut, dim=dim)
